## TUTORIAL 1


In [5]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

df = pd.read_excel('http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')
df.head()

InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom

In [6]:
df['Description'] = df['Description'].str.strip()
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df = df[~df['InvoiceNo'].str.contains('C')]

In [7]:
basket = (df[df['Country'] =="France"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

In [8]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets.drop('POSTAGE', inplace=True, axis=1)

In [9]:
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)

C:\Users\Sergi\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:115: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  DeprecationWarning,


In [10]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

antecedents                   consequents  \
0   (ALARM CLOCK BAKELIKE PINK)  (ALARM CLOCK BAKELIKE GREEN)   
1  (ALARM CLOCK BAKELIKE GREEN)   (ALARM CLOCK BAKELIKE PINK)   
2  (ALARM CLOCK BAKELIKE GREEN)    (ALARM CLOCK BAKELIKE RED)   
3    (ALARM CLOCK BAKELIKE RED)  (ALARM CLOCK BAKELIKE GREEN)   
4   (ALARM CLOCK BAKELIKE PINK)    (ALARM CLOCK BAKELIKE RED)   

   antecedent support  consequent support   support  confidence      lift  \
0            0.102041            0.096939  0.073980    0.725000  7.478947   
1            0.096939            0.102041  0.073980    0.763158  7.478947   
2            0.096939            0.094388  0.079082    0.815789  8.642959   
3            0.094388            0.096939  0.079082    0.837838  8.642959   
4            0.102041            0.094388  0.073980    0.725000  7.681081   

   leverage  conviction  
0  0.064088    3.283859  
1  0.064088    3.791383  
2  0.069932    4.916181  
3  0.069932    5.568878  
4  0.064348    3.293135

In [11]:
rules[ (rules['lift'] >= 6) &
       (rules['confidence'] >= 0.8) ]

antecedents  \
2                        (ALARM CLOCK BAKELIKE GREEN)   
3                          (ALARM CLOCK BAKELIKE RED)   
16                    (SET/6 RED SPOTTY PAPER PLATES)   
18                    (SET/6 RED SPOTTY PAPER PLATES)   
19                      (SET/6 RED SPOTTY PAPER CUPS)   
20  (SET/6 RED SPOTTY PAPER PLATES, SET/6 RED SPOT...   
21  (SET/6 RED SPOTTY PAPER PLATES, SET/20 RED RET...   
22  (SET/6 RED SPOTTY PAPER CUPS, SET/20 RED RETRO...   

                             consequents  antecedent support  \
2             (ALARM CLOCK BAKELIKE RED)            0.096939   
3           (ALARM CLOCK BAKELIKE GREEN)            0.094388   
16  (SET/20 RED RETROSPOT PAPER NAPKINS)            0.127551   
18         (SET/6 RED SPOTTY PAPER CUPS)            0.127551   
19       (SET/6 RED SPOTTY PAPER PLATES)            0.137755   
20  (SET/20 RED RETROSPOT PAPER NAPKINS)            0.122449   
21         (SET/6 RED SPOTTY PAPER CUPS)            0.102041   
22       (SET/6 RED SPOTTY PAPER PLATES)            0.102041   

    consequent support   support  confidence      lift  leverage  conviction  
2             0.094388  0.079082    0.815789  8.642959  0.069932    4.916181  
3             0.096939  0.079082    0.837838  8.642959  0.069932    5.568878  
16            0.132653  0.102041    0.800000  6.030769  0.085121    4.336735  
18            0.137755  0.122449    0.960000  6.968889  0.104878   21.556122  
19            0.127551  0.122449    0.888889  6.968889  0.104878    7.852041  
20            0.132653  0.099490    0.812500  6.125000  0.083247    4.625850  
21            0.137755  0.099490    0.975000  7.077778  0.085433   34.489796  
22            0.127551  0.099490    0.975000  7.644000  0.086474   34.897959

In [12]:
basket2 = (df[df['Country'] =="Germany"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

basket_sets2 = basket2.applymap(encode_units)
basket_sets2.drop('POSTAGE', inplace=True, axis=1)
frequent_itemsets2 = apriori(basket_sets2, min_support=0.05, use_colnames=True)
rules2 = association_rules(frequent_itemsets2, metric="lift", min_threshold=1)

rules2[ (rules2['lift'] >= 4) &
        (rules2['confidence'] >= 0.5)]

C:\Users\Sergi\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:115: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  DeprecationWarning,


antecedents                         consequents  \
1   (PLASTERS IN TIN CIRCUS PARADE)  (PLASTERS IN TIN WOODLAND ANIMALS)   
7        (PLASTERS IN TIN SPACEBOY)  (PLASTERS IN TIN WOODLAND ANIMALS)   
10    (RED RETROSPOT CHARLOTTE BAG)            (WOODLAND CHARLOTTE BAG)   

    antecedent support  consequent support   support  confidence      lift  \
1             0.115974            0.137856  0.067834    0.584906  4.242887   
7             0.107221            0.137856  0.061269    0.571429  4.145125   
10            0.070022            0.126915  0.059081    0.843750  6.648168   

    leverage  conviction  
1   0.051846    2.076984  
7   0.046488    2.011670  
10  0.050194    5.587746

## TUTORIAL 2


In [13]:
dataset = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]

In [14]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

Apple   Corn   Dill   Eggs  Ice cream  Kidney Beans   Milk  Nutmeg  Onion  \
0  False  False  False   True      False          True   True    True   True   
1  False  False   True   True      False          True  False    True   True   
2   True  False  False   True      False          True   True   False  False   
3  False   True  False  False      False          True   True   False  False   
4  False   True  False   True       True          True  False   False   True   

   Unicorn  Yogurt  
0    False    True  
1    False    True  
2    False   False  
3     True    True  
4    False   False

In [15]:
from mlxtend.frequent_patterns import apriori

apriori(df, min_support=0.6)

support   itemsets
0       0.8        (3)
1       1.0        (5)
2       0.6        (6)
3       0.6        (8)
4       0.6       (10)
5       0.8     (3, 5)
6       0.6     (8, 3)
7       0.6     (5, 6)
8       0.6     (8, 5)
9       0.6    (10, 5)
10      0.6  (8, 3, 5)

In [16]:
apriori(df, min_support=0.6, use_colnames=True)

support                     itemsets
0       0.8                       (Eggs)
1       1.0               (Kidney Beans)
2       0.6                       (Milk)
3       0.6                      (Onion)
4       0.6                     (Yogurt)
5       0.8         (Eggs, Kidney Beans)
6       0.6                (Eggs, Onion)
7       0.6         (Milk, Kidney Beans)
8       0.6        (Onion, Kidney Beans)
9       0.6       (Yogurt, Kidney Beans)
10      0.6  (Eggs, Onion, Kidney Beans)

In [17]:
frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

support                     itemsets  length
0       0.8                       (Eggs)       1
1       1.0               (Kidney Beans)       1
2       0.6                       (Milk)       1
3       0.6                      (Onion)       1
4       0.6                     (Yogurt)       1
5       0.8         (Eggs, Kidney Beans)       2
6       0.6                (Eggs, Onion)       2
7       0.6         (Milk, Kidney Beans)       2
8       0.6        (Onion, Kidney Beans)       2
9       0.6       (Yogurt, Kidney Beans)       2
10      0.6  (Eggs, Onion, Kidney Beans)       3

In [18]:
frequent_itemsets[ (frequent_itemsets['length'] == 2) &
                   (frequent_itemsets['support'] >= 0.8) ]

support              itemsets  length
5      0.8  (Eggs, Kidney Beans)       2

In [19]:
frequent_itemsets[ frequent_itemsets['itemsets'] == {'Onion', 'Eggs'} ]

support       itemsets  length
6      0.6  (Eggs, Onion)       2

In [20]:
oht_ary = te.fit(dataset).transform(dataset, sparse=True)
sparse_df = pd.DataFrame.sparse.from_spmatrix(oht_ary, columns=te.columns_)
sparse_df

Apple   Corn   Dill   Eggs  Ice cream  Kidney Beans   Milk  Nutmeg  Onion  \
0  False  False  False   True      False          True   True    True   True   
1  False  False   True   True      False          True  False    True   True   
2   True  False  False   True      False          True   True   False  False   
3  False   True  False  False      False          True   True   False  False   
4  False   True  False   True       True          True  False   False   True   

   Unicorn  Yogurt  
0    False    True  
1    False    True  
2    False   False  
3     True    True  
4    False   False

In [21]:
apriori(sparse_df, min_support=0.6, use_colnames=True, verbose=1)

Processing 21 combinations | Sampling itemset size 3


support                     itemsets
0       0.8                       (Eggs)
1       1.0               (Kidney Beans)
2       0.6                       (Milk)
3       0.6                      (Onion)
4       0.6                     (Yogurt)
5       0.8         (Eggs, Kidney Beans)
6       0.6                (Eggs, Onion)
7       0.6         (Milk, Kidney Beans)
8       0.6        (Onion, Kidney Beans)
9       0.6       (Yogurt, Kidney Beans)
10      0.6  (Eggs, Onion, Kidney Beans)

## Comentarios del ejercicio 

Desde que empezamos a ver este tema se me hizo muy interesante y me llamo la atención porque me pareció que era una manera bastante buena de hacer predicciones, principalmente en los ejemplos vistos como en negocios y mercadeo, pero la verdad pensé que sería un poco más difícil de aplicar a la hora de ya hacerlo en código. Con esta práctica me di cuenta de que, como es costumbre, Python y sus librerías hacen mucho más simple el trabajo y en realidad el algoritmo se emplea de una manera mucho más fácil y rápida de lo que pensaba.

En lo personal me parece que al ser tan simple de aplicar con las librerías de Python, la parte en la que parece que hay que tener más cuidado es en el filtro y limpieza de los datos para que el algoritmo pueda ejecutarse de manera correcta y resolver las predicciones bien. 

Del primer tutorial me gusto mucho ver como el filtro de país cambia mucho los resultados, por que refleja algo que en la vida real pasa bastante, ya que por ejemplo las acciones o preferencias de una persona en Costa Rica pueden varias mucho a lo que hacer o prefiere la gente de países Europeos. Esto también sirve para ver como en un análisis más grande se podría responder muchas incógnitas que tenga el analista de datos con respecto a los factores del data
 set analizado, no siempre tiene que ser país sino talvez edades o género. 
 
También me gusto ver, como es común en Python, las visualizaciones de los datos analizados son bastante fáciles de formatear, ya que tienen muchos atributos que pueden usarse para acomodar mejor y que se pueda interpretar mejor la salida de los datos. Como se puede ver principalmente en los dos últimos ejemplos del tutorial 2.